In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
# استيراد المكتبات
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. تحميل البيانات
data = pd.read_csv("/kaggle/input/data-tweets/Tweets.csv")  # اسم الملف بالضبط

# 2. تنظيف النصوص
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)  # إزالة الروابط
    text = re.sub(r'@\w+', '', text)     # إزالة المنشن
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # إزالة الرموز
    return text

data['clean_tweet'] = data['text'].apply(clean_text)

# 3. تجهيز البيانات
X = data['clean_tweet']
y = data['airline_sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2})  # تحويل المشاعر لأرقام

# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Tokenization & Padding
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# 5. بناء النموذج (LSTM)
model = keras.Sequential([
    keras.layers.Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    keras.layers.LSTM(64, return_sequences=False),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')  # 3 مشاعر
])

# 6. Compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 7. تدريب النموذج
model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_split=0.1)

# 8. تقييم النموذج
y_pred = np.argmax(model.predict(X_test_pad), axis=-1)
print(classification_report(y_test, y_pred, target_names=['negative', 'neutral', 'positive']))


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-09-29 12:49:58.889557: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


165/165 ━━━━━━━━━━━━━━━━━━━━ 14s 60ms/step - accuracy: 0.6007 - loss: 0.9438 - val_accuracy: 0.6152 - val_loss: 0.9298
Epoch 2/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.6237 - loss: 0.9221 - val_accuracy: 0.6152 - val_loss: 0.9313
Epoch 3/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.6196 - loss: 0.9255 - val_accuracy: 0.6152 - val_loss: 0.9316
Epoch 4/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - accuracy: 0.6173 - loss: 0.9259 - val_accuracy: 0.6152 - val_loss: 0.9299
Epoch 5/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - accuracy: 0.6318 - loss: 0.9118 - val_accuracy: 0.6152 - val_loss: 0.9317
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step
              precision    recall  f1-score   support

    negative       0.65      1.00      0.78      1889
     neutral       0.00      0.00      0.00       580
    positive       0.00      0.00      0.00       459

    accuracy                           0.65      2928
   macro avg       0.22      0.33      0.26      2928
w

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
